In [1]:
import numpy as np
import matplotlib.pyplot as plt
from read_nid import *
from PIL import Image
import struct
from ReadWriteSXM import SPM
import pySPM

ModuleNotFoundError: No module named 'pySPM'

In [22]:
import os

In [ ]:
class SM2(pySPM.SXM):
    def __init__(self, filename):
        self.filename = filename
        assert os.path.exists(self.filename)
        self.f = open(self.filename, 'rb')
        self.data_offset = 512
        line = self.f.readline()
        line = str(line)
        lines = line.split()
        # print(f"data_offset: {self.data_offset}")
        self.size = dict(pixels={
            'x': int(lines[7]),
            'y': int(lines[8])
        }, real={
            'x': float(lines[12]) * int(lines[7]),
            'y': float(lines[16]) * int(lines[8]),
            'unit': lines[14]
        })

In [31]:

def read_sxm(file, dontflip=True):
    """
    Gets data from existing SXM file
    :param filename: file
    :param dontflip: flips the matrix as default to match image file
    :return:
    """
    assert os.path.exists(file)
    f = open(file, 'rb')
    l = ''
    key = ''
    header = {}
    while l != b':SCANIT_END:':
        l = f.readline().rstrip()
        if l[:1] == b':':
            key = l.split(b':')[1].decode('ascii')
            header[key] = []
        else:
            if l:  # remove empty lines
                try:
                    header[key].append(l.decode('ascii').split())
                except KeyError as e:
                    print(f"KeyError: {key}, probably wrong filetype")
    while f.read(1) != b'\x1a':
        pass
    assert f.read(1) == b'\x04'
    assert header['SCANIT_TYPE'][0][0] in ['FLOAT', 'INT', 'UINT', 'DOUBLE']
    data_offset = f.tell()
    size = dict(pixels={
        'x': int(header['SCAN_PIXELS'][0][0]),
        'y': int(header['SCAN_PIXELS'][0][1])
    }, real={
        'x': float(header['SCAN_RANGE'][0][0]),
        'y': float(header['SCAN_RANGE'][0][1]),
        'unit': 'm'
    })
    im_size = size['pixels']['x'] * size['pixels']['y']
    data = np.array(struct.unpack('<>'['MSBFIRST' == header['SCANIT_TYPE'][0][1]] + str(im_size) +
                                  {'FLOAT': 'f', 'INT': 'i', 'UINT': 'I', 'DOUBLE': 'd'}[
                                      header['SCANIT_TYPE'][0][0]],
                                  f.read(4 * im_size))).reshape((size['pixels']['y'], size['pixels']['x']))

    x = data[0][0]


    for i in range(data.shape[0]):
        if np.isnan(data[i, 0]) or np.isnan(data[i, -1]):
            continue
        data = data[i:, :]
        break


    if not dontflip:
        data = np.flipud(data)

    mini = np.amin(data)
    maxi = np.amax(data)

    data = data.astype(float)
    data = (data - mini) / (maxi - mini)


    return data


In [32]:
def read_spm(file):
    """
    Read .spm file and save result as grayscale png
    :param file:
    :return:
    """
    # print("ReadSPM ", file, " -> ", resultf)
    chID = 0  # 0: Image, 1: DI/DV, 2: ?
    spm = SPM(file)
    dat = spm.get_data(chID=chID)
    dat = dat.astype(float)
    maxi = np.amax(dat)
    mini = np.amin(dat)
    dat = (dat - mini) / (maxi - mini)
    dat *= 255
    dat = dat.astype(np.uint8)
    dat = np.flipud(dat)
    # dat = np.fliplr(dat)
    return dat

In [23]:
nid_file = r"D:\Dateien\KI_Speicher\SampleSXM\FormatComparison\Image00085.nid"
sxm_file = r"D:\Dateien\KI_Speicher\SampleSXM\FormatComparison\Au-Au094.sxm"
sm2_file = r"D:\Dateien\KI_Speicher\SampleSXM\FormatComparison\Image001255.SM2"
bmp_file = r"D:\Dateien\KI_Speicher\SampleSXM\FormatComparison\Au-Au094.bmp"

In [24]:
nid_dat = read(nid_file)

In [25]:
nid_arr = nid_dat.data[2]

In [29]:
bmp_arr = np.array(Image.open(bmp_file))[:, :, 0]

In [37]:
sxm_arr = read_sxm(sxm_file)

In [43]:
spm_arr = read_spm(sm2_file)

Exception:  Probably wrong filetype, maybe errorbar set to low
Invalid file  D:\Dateien\KI_Speicher\SampleSXM\FormatComparison\Image001255.SM2


AttributeError: 'SPM' object has no attribute 'size'